<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Exploratory-Data-Analysis-and-Metric" data-toc-modified-id="Exploratory-Data-Analysis-and-Metric-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Exploratory Data Analysis and Metric</a></span></li><li><span><a href="#Model" data-toc-modified-id="Model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Model</a></span></li><li><span><a href="#Submit" data-toc-modified-id="Submit-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Submit</a></span></li><li><span><a href="#Homework" data-toc-modified-id="Homework-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Homework</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Часть-1.-(4-балла)" data-toc-modified-id="Часть-1.-(4-балла)-4.0.1"><span class="toc-item-num">4.0.1&nbsp;&nbsp;</span>Часть 1. (4 балла)</a></span></li><li><span><a href="#Часть-2.-(3-баллов)" data-toc-modified-id="Часть-2.-(3-баллов)-4.0.2"><span class="toc-item-num">4.0.2&nbsp;&nbsp;</span>Часть 2. (3 баллов)</a></span></li></ul></li></ul></li></ul></div>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing  import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import balanced_accuracy_score

In [ ]:
SEED = 42

In [ ]:
DATA_PATH = './data/competition_data'

**[Kaggle InClass: Language Detection](https://www.kaggle.com/t/6b23f7b92b4b497e9aead739dd00ae1a)**

# Exploratory Data Analysis and Metric

In [ ]:
train_df = pd.read_csv(f'{DATA_PATH}/train.csv.gz')
test_df = pd.read_csv(f'{DATA_PATH}/test.csv.gz')

In [ ]:
train_df.shape, test_df.shape

In [ ]:
train_df.head(15)

In [ ]:
lang_count = (
    train_df.language.value_counts()
    .to_frame()
    .reset_index()
)

lang_count.head()

In [ ]:
fig, ax = plt.subplots(figsize=(8,25))

ax.barh(lang_count['index'], lang_count['language'])
ax.invert_yaxis();

In [ ]:
lang_count[lang_count['index'].apply(lambda x: x in {"en", "kk"})]

Датасет сильно не сбалансирован по классам, а требуется хорошее качество для всех языков, поэтому метрика в данном соревновании — взвешенная доля правильных ответов:

$$ 
\text{WeightedAccuracy}\left(y, \hat{y}, w\right) = \frac{\displaystyle{\sum_{i=1}^{n}} w_i \cdot \left(y_i == \hat{y}_i\right)}{\displaystyle\sum_{i=1}^{n} w_i } 
$$

Веса выбирались обратно пропорционально количеству примеров соответствующего языка:
$$
w\left(\text{"en"}\right) \sim \frac{1}{157655}
$$
$$ \dots $$
$$
w\left(\text{"kk"}\right) \sim \frac{1}{2046}
$$

Поэтому для валидации можно использовать [sklearn.metrics.balanced_accuracy_score](https://scikit-learn.org/stable/modules/model_evaluation.html#balanced-accuracy-score)

# Model

In [ ]:
x = train_df.values[:,  0]
label_encoder = LabelEncoder().fit(train_df.values[:, 1])
y = label_encoder.transform(train_df.values[:, 1])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=SEED)

In [ ]:
pipe = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('model', SGDClassifier(random_state=SEED, loss='log'))
])

In [ ]:
pipe.fit(x_train, y_train)

In [ ]:
predictions = pipe.predict(x_train)

In [ ]:
balanced_accuracy_score(y_train, predictions)

In [ ]:
test_predictions = pipe.predict(x_test)

In [ ]:
balanced_accuracy_score(y_test, test_predictions)

# Submit

In [ ]:
submit_predictions = pipe.predict(test_df.sentence.values)

In [ ]:
test_df['language'] = label_encoder.classes_[submit_predictions]

test_df.head()

In [ ]:
(
    test_df[['index',  'language']]
    .to_csv(f'{DATA_PATH}/sample_submission.csv', index=False)
)

In [ ]:
!head $DATA_PATH/sample_submission.csv

# Homework

### Часть 1. (4 балла)
- исследовать данные
    - почистить обучающую выборку
        - есть ли ошибки в разметке?
    - посмотреть, какие символы / токены встречаются во всех языках
    - выделить коллокации для известных вам языков (хотя бы русский и английский)
- попробовать добавить в модель признаки на основе word2vec или topic modeling
- построить confusion_matrix и посмотреть, какие языки путает модель
- вывести top_k токенов с максимальной важностью для каждого класса
 
### Часть 2. (3 баллов)
- выбить хороший score на private leaderboard'е